In [0]:
from google.colab import files

uploaded = files.upload()

Saving spam.csv to spam.csv


In [0]:
import io
import pandas as pd


data = pd.read_csv(io.BytesIO(uploaded['spam.csv']), encoding = 'latin-1')
data.head()
data.rename({'v1':'label','v2':'text'}, axis = 1, inplace = True)
data.drop(columns = ['Unnamed: 2','Unnamed: 3','Unnamed: 4'], inplace = True)
data['text'] = data.text.astype(str)

In [0]:
spam_data = data[data['label'] == 'spam']['text']
type(spam_data)
len(spam_data)

747

In [0]:
!pip install pyldavis

In [0]:
import pandas as pd
import numpy as np
import nltk 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.model_selection import GridSearchCV
from pprint import pprint
import spacy
import gensim
from gensim import models, corpora

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [0]:
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in spam_data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

data[10]

'SMS. ac Sptv: The New Jersey Devils and the Detroit Red Wings play Ice Hockey. Correct or Incorrect? End? Reply END SPTV'

In [0]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))


In [0]:
import spacy.cli
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
spacy.cli.download('en_core_web_lg')

nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])


In [0]:
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['free entry wkly comp win cup final tkts may text receive entry question std txt rate cs apply', 'freemsg there darl  be week now word back would fun still tb xxx std chgs send rcv']


In [0]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )


data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [0]:
# Define Search Param
search_params = {'n_components': [4, 5, 6, 7, 8, 9, 10], 'learning_decay': [.4,.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_components': [4, 5, 6, 7, 8, 9, 10], 'learning_decay': [0.4, 0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [0]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)


Best Model's Params:  {'learning_decay': 0.7, 'n_components': 4}


In [0]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic


df_document_topic.head()

,Topic0,Topic1,Topic2,Topic3,dominant_topic
Doc0,0.02,0.78,0.02,0.18,1
Doc1,0.04,0.04,0.04,0.89,3
Doc2,0.02,0.93,0.03,0.02,1
Doc3,0.02,0.94,0.02,0.02,1
Doc4,0.24,0.31,0.02,0.42,3


In [0]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,3,298
1,0,177
2,2,139
3,1,133


In [0]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
3      34.490634        1       1    4.706093  191.011322
2      22.530559        1       2 -352.118500 -230.997910
0      22.074523        1       3   69.858070 -587.861328
1      20.904284        1       4  426.682281 -165.852036, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
132   Default  106.000000       reply  106.000000  30.0000  30.0000
166   Default   62.000000      urgent   62.000000  29.0000  29.0000
155   Default  139.000000        text  139.000000  28.0000  28.0000
150   Default  120.000000        stop  120.000000  27.0000  27.0000
141   Default   70.000000     service   70.000000  26.0000  26.0000
121   Default   44.000000         ppm   44.000000  25.0000  25.0000
8     Default   67.000000       award   67.000000  24.0000  24.0000
129   Default   38.000000     receive   38.000000  23.0000  23.0000
19    Default  114.000000       claim  114.000000  22.0000  22.0000
178   Default  143.000000         win  143.000000  21.0000  21.0000
10    Default   56.000000         box   56.000000  20.0000  20.0000
140   Default   78.000000        send   78.000000  19.0000  19.0000
124   Default   95.000000       prize   95.000000  18.0000  18.0000
35    Default   50.000000    customer   50.000000  17.0000  17.0000
139   Default   27.000000      select   27.000000  16.0000  16.0000
4     Default   30.000000       apply   30.000000  15.0000  15.0000
119   Default   27.000000        poly   27.000000  14.0000  14.0000
23    Default   26.000000  collection   26.000000  13.0000  13.0000
76    Default   25.000000        know   25.000000  12.0000  12.0000
183   Default   96.000000         www   96.000000  11.0000  11.0000
53    Default  220.000000        free  220.000000  10.0000  10.0000
14    Default   75.000000        cash   75.000000   9.0000   9.0000
25    Default   56.000000         com   56.000000   8.0000   8.0000
56    Default   27.000000        game   27.000000   7.0000   7.0000
65    Default   32.000000     holiday   32.000000   6.0000   6.0000
117   Default   41.000000       pobox   41.000000   5.0000   5.0000
165   Default   22.000000      update   22.000000   4.0000   4.0000
68    Default   22.000000        http   22.000000   3.0000   3.0000
79    Default   37.000000        late   37.000000   2.0000   2.0000
67    Default   22.000000         hrs   22.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
169    Topic4    9.318851       value   11.928956   1.3183  -4.8633
84     Topic4   11.343399        make   14.895905   1.2928  -4.6667
134    Topic4   12.337781      reveal   17.814861   1.1978  -4.5827
180    Topic4   10.252491        wkly   14.844186   1.1951  -4.7679
26     Topic4    7.356845        comp   10.847421   1.1769  -5.0998
48     Topic4   18.688348       entry   28.538969   1.1418  -4.1675
108    Topic4   11.290008         opt   17.723118   1.1143  -4.6715
8      Topic4   42.998636       award   67.732600   1.1108  -3.3342
85     Topic4    9.142113       match   15.682652   1.0256  -4.8825
13     Topic4   19.889815      camera   35.790680   0.9777  -4.1052
115    Topic4    7.080929      player   12.781675   0.9746  -5.1380
79     Topic4   20.617779        late   37.771445   0.9598  -4.0692
100    Topic4    8.506261         net   15.646134   0.9558  -4.9546
105    Topic4   21.170583      number   39.624213   0.9384  -4.0428
19     Topic4   50.837368       claim  114.446324   0.7537  -3.1668
183    Topic4   40.023417         www   96.567629   0.6844  -3.4059
25     Topic4   24.198173         com   56.302171   0.7208  -3.9091
178    Topic4   42.138777         win  143.434747   0.3403  -3.3544
92     Topic4   40.690651      mobile  139.477705   0.3333  -3.3894
53     Topic4   47.694703        free  220.817829   0.0327  -3.2306
163  

In [0]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

In [0]:
# Show top n keywords for each topic
def show_topics(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,win,service,mobile,prize,cash,send,customer,poly,game,know,ringtone,claim,text,txt,chat
Topic 1,claim,free,award,win,mobile,www,txt,receive,apply,select,com,update,http,number,late
Topic 2,urgent,ppm,prize,claim,tone,box,nokia,mobile,free,win,contact,holiday,min,txt,collection
Topic 3,free,text,stop,reply,txt,new,send,www,min,just,message,msg,week,sms,com


In [0]:
# Define function to predict topic for a given text document.
#nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

def predict_topic(text, nlp=nlp):
    global sent_to_words
    global lemmatization

    # Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))

    # Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Step 3: Vectorize transform
    mytext_4 = vectorizer.transform(mytext_3)

    # Step 4: LDA Transform
    topic_probability_scores = best_lda_model.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    return topic, topic_probability_scores

# Predict the topic
mytext = ["Some text about christianity and bible"]
topic, prob_scores = predict_topic(text = mytext)
print(topic, prob_scores)

best_lda_model.transform(vectorizer.transform(mytext))[0]

array([0.12828229, 0.12500221, 0.12500184, 0.62171366])